In [2]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col,first, countDistinct, isnan, when, count, round, substring_index,substring, split, regexp_replace, udf
from tabulate import tabulate
import pandas as pd



 ## Since we write local [*] in the master, it will use all cores in our machine. If we said local [4] it will work with 4 cores.



 ## getOrCreate is used to create a SparkSession if not present.

In [3]:
print_reports = False

In [4]:

spark=SparkSession.builder\
    .master("local[*]")\
    .appName("LoanApproval")\
    .getOrCreate()


In [5]:

sc=spark.sparkContext


 ## Read Data - SBAnational.csv

In [6]:

data_path="../data/SBAnational.csv"


In [7]:

loan_df =  spark.read.csv(data_path, header=True, inferSchema=True, quote='"', escape='"', multiLine=True)


In [8]:

loan_df.show(5)
print('=====================')
print("Number of rows in the dataframe:")
print('=====================')
loan_df_count = loan_df.count()
print(loan_df_count)
print('=====================')
print("Number of columns in the dataframe:")
print('=====================')
print(len(loan_df.columns))
print('=====================')
print("Schema of the dataframe:")
print('=====================')
loan_df.printSchema() #prints the dataframe schema
print('=====================')
print("Columns in the dataframe:")
print('=====================')
print(loan_df.columns) 


+-------------+--------------------+------------+-----+-----+--------------------+---------+------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------------+------------+
|LoanNr_ChkDgt|                Name|        City|State|  Zip|                Bank|BankState| NAICS|ApprovalDate|ApprovalFY|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|      GrAppv|    SBA_Appv|
+-------------+--------------------+------------+-----+-----+--------------------+---------+------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------------+------------+
|   1000014003|      ABC HOBBYCRAFT|  EVANSVILLE|

In [9]:
loan_df.describe().show()

+-------+--------------------+---------------+------------------+------+------------------+--------------------+---------+------------------+------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+--------------------+----------+----------------+-----------------+------------+----------+------------+--------------+--------------+
|summary|       LoanNr_ChkDgt|           Name|              City| State|               Zip|                Bank|BankState|             NAICS|ApprovalDate|        ApprovalFY|              Term|             NoEmp|          NewExist|         CreateJob|       RetainedJob|     FranchiseCode|        UrbanRural|           RevLineCr|              LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|        GrAppv|      SBA_Appv|
+-------+--------------------+---------------+------------------+------+----

 # Preprocessing and cleaning

 ### Report

In [10]:
# =========================================================================
# =========================================================================
# ============================= DF REPORT =================================
# =========================================================================
# =========================================================================
def report_df(df, header):
    # Calculate the total number of rows
    rdd_count = df.count()

    # Initialize lists to store column statistics
    col_names = []
    data_types = []
    unique_samples = []
    num_uniques = []
    nan_percentages = []
    report_data = []

    # Iterate over each column
    for col_name in header:
        print(col_name)
        # Append column name
        col_names.append(col_name)
        selected_col = col(col_name)
        selected_col_df = df.select(selected_col)

        # Determine data type
        dtype = selected_col_df.dtypes[0][1]
        data_types.append(dtype)
        distinct_df = selected_col_df.distinct()
        # Collect unique values
        unique_sample = [row[col_name] for row in distinct_df.limit(2).collect()]
        unique_samples.append(unique_sample)

        # Count number of unique values
        n_unique = distinct_df.count()
        num_uniques.append(n_unique)

        # Calculate percentage of NaN values
        none_percentage_val = df.filter(selected_col.isNull()).count() / rdd_count * 100
        nan_percentages.append(none_percentage_val)
        report_data.append([col_name, dtype, unique_sample, n_unique, none_percentage_val])

    return report_data


In [11]:
if print_reports:
    report_res = report_df(loan_df, loan_df.columns)
    # Display the result
    column_names = ['Column', 'Type', 'Unique Sample', 'N Unique', '%None']
    print(tabulate(report_res, headers=column_names, tablefmt='grid'))




In [12]:

def show_percentage_of_each_value_in_column(df, df_count,col_name,show_num=10):
    # Calculate percentage of 0s and 1s
    percentage_df = df.groupBy(col_name).agg((count("*") / df_count).alias("Percentage"))

    # Round percentage values to two decimal places
    percentage_df = percentage_df.withColumn("Percentage", round(col("Percentage") * 100, 2))

    # sort the dataframe by percentage descending
    percentage_df = percentage_df.sort(col("Percentage").desc())

    # Show result
    percentage_df.show(show_num)

def show_df_where_col_isnull(df, col_name, show_num):
    # Filter rows where 'Name' column is null
    filtered_df = df.filter(col(col_name).isNull())

    # Show the resulting DataFrame
    filtered_df.show(show_num)
    null_count = filtered_df.count()
    print(f"Null Count: {null_count}")

def print_unique_val_num_in_col(df, col_name):
    # percentage of unique values in the city
    unique_count = loan_df.select(col_name).distinct().count()
    print(f"Number of unique values in {col_name}: {unique_count}")
    percentage = unique_count / loan_df_count * 100
    print(f"Percentage of unique values in {col_name}: {percentage:.2f}%")

 ### 1. LoanNr_ChkDgt - ID

 Drop the column as it is an ID column and does not provide any information for the analysis.

In [13]:

loan_df = loan_df.drop('LoanNr_ChkDgt')


 ### 2. Name - Name of Borrower

 Drop the column as it is a name column and does not provide any information for the analysis.

In [14]:

col_name = 'Name'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name, show_num=5)

+-------------------+----------+
|               Name|Percentage|
+-------------------+----------+
|             SUBWAY|      0.14|
|      QUIZNO'S SUBS|      0.05|
|        DAIRY QUEEN|      0.04|
|COLD STONE CREAMERY|      0.04|
|      THE UPS STORE|      0.04|
+-------------------+----------+
only showing top 5 rows



In [15]:
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+----+----------+-----+-----+--------------------+---------+------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+-----------+-----------+
|Name|      City|State|  Zip|                Bank|BankState| NAICS|ApprovalDate|ApprovalFY|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|     GrAppv|   SBA_Appv|
+----+----------+-----+-----+--------------------+---------+------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+-----------+-----------+
|NULL|      TROY|   NY|12180|MANUFACTURERS & T...|       NY|     0|   16-Oct-98|      1999|  60|    2|       1|        0|          0|            1|    

In [16]:
# Fill null values in the 'Name' column with 'Unknown Company'
loan_df = loan_df.fillna({col_name: 'Unknown Name'})
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+----+----+-----+---+----+---------+-----+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+
|Name|City|State|Zip|Bank|BankState|NAICS|ApprovalDate|ApprovalFY|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|GrAppv|SBA_Appv|
+----+----+-----+---+----+---------+-----+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+
+----+----+-----+---+----+---------+-----+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+


In [17]:

print_unique_val_num_in_col(loan_df, col_name)


Number of unique values in Name: 779587
Percentage of unique values in Name: 86.70%


 Drop as most of the names are unique

In [18]:

# loan_df = loan_df.drop('Name')


 ### 3. City - City of Borrower



In [19]:

# Count the occurrences of each value in city column
col_name = 'City'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name, show_num=5)


+-----------+----------+
|       City|Percentage|
+-----------+----------+
|LOS ANGELES|      1.29|
|    HOUSTON|      1.14|
|   NEW YORK|      0.87|
|    CHICAGO|      0.67|
|      MIAMI|      0.62|
+-----------+----------+
only showing top 5 rows



In [20]:
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+--------------------+----+-----+-----+--------------------+---------+-----+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------------+------------+
|                Name|City|State|  Zip|                Bank|BankState|NAICS|ApprovalDate|ApprovalFY|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|      GrAppv|    SBA_Appv|
+--------------------+----+-----+-----+--------------------+---------+-----+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------------+------------+
|  BUSATH PHOTOGRAPHY|NULL|   UT|84109|MOUNTAIN W. SMALL...|       UT|    0|   15-Dec-81|      1982| 300|    2|       

In [21]:
# Fill null values in the 'Name' column with 'Unknown Company'
loan_df = loan_df.fillna({col_name: 'Unknown City'})
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+----+----+-----+---+----+---------+-----+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+
|Name|City|State|Zip|Bank|BankState|NAICS|ApprovalDate|ApprovalFY|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|GrAppv|SBA_Appv|
+----+----+-----+---+----+---------+-----+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+
+----+----+-----+---+----+---------+-----+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+


In [22]:
print_unique_val_num_in_col(loan_df, col_name)

Number of unique values in City: 32582
Percentage of unique values in City: 3.62%


In [23]:

unique_city_df = loan_df.select(col_name).groupBy(col_name).agg((count("*")).alias("Count")).sort(col("Count").desc())
unique_city_df.show()


+--------------+-----+
|          City|Count|
+--------------+-----+
|   LOS ANGELES|11558|
|       HOUSTON|10247|
|      NEW YORK| 7846|
|       CHICAGO| 6036|
|         MIAMI| 5594|
|     SAN DIEGO| 5363|
|        DALLAS| 5085|
|       PHOENIX| 4493|
|     LAS VEGAS| 4390|
|   SPRINGFIELD| 3738|
|      BROOKLYN| 3728|
|        DENVER| 3550|
|   SAN ANTONIO| 3515|
|SALT LAKE CITY| 3511|
|        AUSTIN| 3499|
|       SEATTLE| 3470|
| SAN FRANCISCO| 3365|
|      PORTLAND| 3193|
|      COLUMBUS| 3186|
|  PHILADELPHIA| 3178|
+--------------+-----+
only showing top 20 rows



 ### 4. State - State of Borrower

In [24]:

col_name = 'State'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+-----+----------+
|State|Percentage|
+-----+----------+
|   CA|     14.53|
|   TX|      7.84|
|   NY|      6.42|
|   FL|      4.58|
|   PA|      3.91|
|   OH|      3.63|
|   IL|       3.3|
|   MA|      2.81|
|   MN|      2.71|
|   NJ|      2.67|
+-----+----------+
only showing top 10 rows



In [25]:
print_unique_val_num_in_col(loan_df, col_name)


Number of unique values in State: 52
Percentage of unique values in State: 0.01%


In [26]:
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+--------------------+------------------+-----+-----+--------------------+---------+------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------------+------------+
|                Name|              City|State|  Zip|                Bank|BankState| NAICS|ApprovalDate|ApprovalFY|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|      GrAppv|    SBA_Appv|
+--------------------+------------------+-----+-----+--------------------+---------+------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------------+------------+
|SO. JERSEY DANCE/...|        PENNSVILLE| NULL| 8070|                NULL

In [27]:
# Filter rows where 'State' column is null
filtered_df = loan_df.filter(col(col_name).isNull())

# Extract unique values from 'Zip' column
unique_zips = filtered_df.select('Zip').distinct().collect()

# Extract unique zip codes as a list
unique_zip_list = [row['Zip'] for row in unique_zips]

# Print unique zip codes
print(unique_zip_list)

[54205, 76052, 76645, 67219, 33410, 75236, 84124, 79925, 8070, 95682, 65049, 96205, 54025, 0]


In [28]:
# Sort the DataFrame by 'Zip Code' in ascending order
df_sorted = loan_df.orderBy('Zip')

# Define a window specification for the group
window_spec = Window.partitionBy('Zip').orderBy('Zip')

# Fill the null 'State' values with the corresponding non-null 'State' value within each group
loan_df = df_sorted.withColumn(col_name, first(col_name, ignorenulls=True).over(window_spec))

# Show the resulting DataFrame
loan_df.show()

+--------------------+------------------+-----+---+--------------------+---------+------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+--------------+------------+
|                Name|              City|State|Zip|                Bank|BankState| NAICS|ApprovalDate|ApprovalFY|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|        GrAppv|    SBA_Appv|
+--------------------+------------------+-----+---+--------------------+---------+------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+--------------+------------+
| Grass Valley Market|      Grass Valley|   OR|  0|WELLS FARGO BANK ...| 

In [29]:
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+-----------------+----------------+-----+-----+--------------------+---------+------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+-----------+-----------+
|             Name|            City|State|  Zip|                Bank|BankState| NAICS|ApprovalDate|ApprovalFY|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|     GrAppv|   SBA_Appv|
+-----------------+----------------+-----+-----+--------------------+---------+------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+-----------+-----------+
|THE COMPUTER EDGE|BOX 267, APO  AP| NULL|96205|RTC/WESTPORT SAVI...|       CA|541511|   21-Ja

Only 1 state with null values exist, so it is easy to look it up using its zip code.
Zip code 96205 exists in AP.

In [30]:
loan_df = loan_df.fillna({col_name: 'AP'})
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+----+----+-----+---+----+---------+-----+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+
|Name|City|State|Zip|Bank|BankState|NAICS|ApprovalDate|ApprovalFY|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|GrAppv|SBA_Appv|
+----+----+-----+---+----+---------+-----+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+
+----+----+-----+---+----+---------+-----+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+


 ### 5. Zip - Zip code of Borrower

In [31]:

col_name = 'Zip'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+-----+----------+
|  Zip|Percentage|
+-----+----------+
|10001|       0.1|
|90015|       0.1|
|93401|      0.09|
|90010|      0.08|
|33166|      0.07|
|90021|      0.07|
|65804|      0.07|
|59601|      0.07|
|92069|      0.06|
|84107|      0.06|
+-----+----------+
only showing top 10 rows



In [32]:
print_unique_val_num_in_col(loan_df, col_name)

Number of unique values in Zip: 33611
Percentage of unique values in Zip: 3.74%


Least zip code in the US starts with 502, so any value less than this is unvalid.

In [33]:
# Filter rows where 'Zip' column is less than 501
filtered_df = loan_df.filter(col('Zip') < 501)

# Extract unique values from the filtered 'Zip' column
unique_zips = filtered_df.select('Zip').distinct().collect()
all_zips = filtered_df.select('Zip').collect()

# Extract unique zip codes as a list
unique_zip_list = [row['Zip'] for row in unique_zips]
all_zip_list = [row['Zip'] for row in all_zips]

# Print unique zip codes
print("Unique Zip codes less than 501:")
print(unique_zip_list)
print(f"Count: {len(all_zip_list)}")

Unique Zip codes less than 501:
[211, 417, 128, 1, 182, 6, 3, 432, 5, 92, 207, 9, 290, 301, 4, 8, 7, 465, 438, 459, 38, 477, 98, 345, 204, 260, 2, 0]
Count: 400


In [34]:
# Filter rows where 'Zip' column is less than 501
loan_df = loan_df.filter(loan_df[col_name] >= 501)

In [35]:

unique_zip_df = loan_df.select(col_name).groupBy(col_name).agg((count("*")).alias("Count")).sort(col("Count").desc())
unique_zip_df.show()


+-----+-----+
|  Zip|Count|
+-----+-----+
|10001|  933|
|90015|  926|
|93401|  806|
|90010|  733|
|33166|  671|
|90021|  666|
|59601|  640|
|65804|  599|
| 3801|  581|
|59101|  578|
|84115|  577|
|92121|  567|
|92101|  565|
|90670|  531|
|11354|  530|
|59102|  519|
|85260|  518|
|84107|  513|
|89102|  500|
|92069|  499|
+-----+-----+
only showing top 20 rows



Cast Zip to string to treat is as a categorical feature.

In [36]:
# Cast the 'Zip' column to string
loan_df = loan_df.withColumn('Zip', col('Zip').cast('string'))

In [37]:

# loan_df = loan_df.drop(col_name)


 ### 6. Bank - Name of the bank that gave the loan

In [38]:

col_name = 'Bank'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)

+--------------------+----------+
|                Bank|Percentage|
+--------------------+----------+
|BANK OF AMERICA N...|      9.66|
|WELLS FARGO BANK ...|      7.06|
|JPMORGAN CHASE BA...|      5.35|
|U.S. BANK NATIONA...|      3.91|
|CITIZENS BANK NAT...|       3.9|
|PNC BANK, NATIONA...|      3.04|
|           BBCN BANK|      2.55|
|CAPITAL ONE NATL ...|      2.47|
|MANUFACTURERS & T...|      1.25|
|READYCAP LENDING,...|      1.19|
+--------------------+----------+
only showing top 10 rows



In [39]:
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+--------------------+-------------+-----+----+----+---------+------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------------+------------+
|                Name|         City|State| Zip|Bank|BankState| NAICS|ApprovalDate|ApprovalFY|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|      GrAppv|    SBA_Appv|
+--------------------+-------------+-----+----+----+---------+------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------------+------------+
|BERKSHIRE HARDWOO...| CHESTERFIELD|   MA|1012|NULL|     NULL|     0|    5-Dec-89|      1990| 180|   14|       1|        0|          0|    

In [40]:
print_unique_val_num_in_col(loan_df, col_name)

Number of unique values in Bank: 5803
Percentage of unique values in Bank: 0.65%


In [41]:
# Fill null values in the 'Bank' column with 'Unknown Bank'
loan_df = loan_df.fillna({col_name: 'Unknown Bank'})
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+----+----+-----+---+----+---------+-----+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+
|Name|City|State|Zip|Bank|BankState|NAICS|ApprovalDate|ApprovalFY|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|GrAppv|SBA_Appv|
+----+----+-----+---+----+---------+-----+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+
+----+----+-----+---+----+---------+-----+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+


 ### 7. BankState - State of Bank

In [42]:

col_name = 'BankState'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+---------+----------+
|BankState|Percentage|
+---------+----------+
|       CA|     13.13|
|       NC|      8.84|
|       IL|      7.33|
|       OH|       6.5|
|       SD|      5.68|
|       TX|      5.31|
|       RI|      5.04|
|       NY|       4.4|
|       VA|      3.22|
|       DE|      2.73|
+---------+----------+
only showing top 10 rows



In [43]:
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+--------------------+-------------+-----+----+------------+---------+------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------------+------------+
|                Name|         City|State| Zip|        Bank|BankState| NAICS|ApprovalDate|ApprovalFY|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|      GrAppv|    SBA_Appv|
+--------------------+-------------+-----+----+------------+---------+------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------------+------------+
|BERKSHIRE HARDWOO...| CHESTERFIELD|   MA|1012|Unknown Bank|     NULL|     0|    5-Dec-89|      1990| 180|   14|   

In [44]:
print_unique_val_num_in_col(loan_df, col_name)

Number of unique values in BankState: 57
Percentage of unique values in BankState: 0.01%


Drop nulls as we cant populate them.

In [45]:
loan_df = loan_df.dropna(subset=[col_name])

In [120]:
show_df_where_col_isnull(loan_df, "BankState", show_num=5)

+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+----------+---------+------+----------+------+-------------+-----------+------------+
|Name|City|State|Zip|Bank|BankState|Term|NoEmp|NewExist|CreateJob|RetainedJob|UrbanRural|RevLineCr|LowDoc|MIS_Status|Sector|ApprovalMonth|IsFranchise|clean_GrAppv|
+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+----------+---------+------+----------+------+-------------+-----------+------------+
+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+----------+---------+------+----------+------+-------------+-----------+------------+

Null Count: 0


 ### 8. NAICS - North American Industry Classification System code for the industry where the business is located

In [46]:

col_name='NAICS'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)

+------+----------+
| NAICS|Percentage|
+------+----------+
|     0|     22.34|
|722110|      3.11|
|722211|      2.16|
|811111|      1.62|
|621210|      1.56|
|624410|      1.12|
|812112|      1.03|
|561730|      0.99|
|621310|      0.97|
|812320|      0.88|
+------+----------+
only showing top 10 rows



In [47]:

print_unique_val_num_in_col(loan_df, col_name)

Number of unique values in NAICS: 1312
Percentage of unique values in NAICS: 0.15%


In [48]:
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+----+----+-----+---+----+---------+-----+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+
|Name|City|State|Zip|Bank|BankState|NAICS|ApprovalDate|ApprovalFY|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|GrAppv|SBA_Appv|
+----+----+-----+---+----+---------+-----+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+
+----+----+-----+---+----+---------+-----+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+


In [49]:

# Convert NAICS code into related sector

# Extract first two characters of NAICS code
first_two_chars = substring(loan_df["NAICS"], 1, 2)
# print(first_two_chars)[0]

# Apply mapping using when and otherwise
loan_df = loan_df.withColumn("Sector",
    first_two_chars
)
loan_df = loan_df.drop("NAICS")
col_name='Sector'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+------+----------+
|Sector|Percentage|
+------+----------+
|     0|     22.34|
|    44|      9.41|
|    81|      8.07|
|    54|      7.57|
|    72|      7.52|
|    23|       7.4|
|    62|      6.15|
|    42|      5.41|
|    45|      4.72|
|    33|      4.25|
+------+----------+
only showing top 10 rows



In [50]:

naics_to_sector = {
    11: 'Agriculture, Forestry, Fishing and Hunting',
    21: 'Mining, Quarrying, and Oil and Gas Extraction',
    22: 'Utilities',
    23: 'Construction',
    31: 'Manufacturing',
    32: 'Manufacturing',
    33: 'Manufacturing',
    42: 'Wholesale Trade',
    44: 'Retail Trade',
    45: 'Retail Trade',
    48: 'Transportation and Warehousing',
    49: 'Transportation and Warehousing',
    51: 'Information',
    52: 'Finance and Insurance',
    53: 'Real Estate and Rental and Leasing',
    54: 'Professional, Scientific, and Technical Services',
    55: 'Management of Companies and Enterprises',
    56: 'Administrative and Support and Waste Management and Remediation Services',
    61: 'Educational Services',
    62: 'Health Care and Social Assistance',
    71: 'Arts, Entertainment, and Recreation',
    72: 'Accommodation and Food Services',
    81: 'Other Services (except Public Administration)',
    92: 'Public Administration'
}

loan_df = loan_df.withColumn(col_name, 
                   when(col(col_name) == 32, 31)
                   .when(col(col_name) == 33, 31)
                   .when(col(col_name) == 45, 44)
                   .when(col(col_name) == 49, 48))
# Cast the 'Zip' column to string
# loan_df = loan_df.withColumn('Sector', col('Sector').cast('string'))
# # Convert NAICS codes to their corresponding sectors
# loan_df_temp = loan_df.withColumn(col_name, 
#                    when(col(col_name).isin(list(naics_to_sector.keys())), 
#                         naics_to_sector[col(col_name).cast("int")])  # Cast to int before accessing dictionary
#                    .otherwise("Unknown"))

# loan_df_temp.show(5)


In [51]:
loan_df =loan_df.dropna(subset=["Sector"]) 

 ### 9. ApprovalDate - Date SBA commitment issued

In [52]:

col_name = 'ApprovalDate'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+------------+----------+
|ApprovalDate|Percentage|
+------------+----------+
|    5-Sep-03|      0.01|
|   14-Oct-05|      0.01|
|   22-Oct-04|      0.01|
|   17-Jul-03|      0.01|
|   14-Nov-03|      0.01|
|    9-Feb-05|      0.01|
|   30-Jul-04|      0.01|
|   30-Nov-06|      0.01|
|    2-Feb-06|      0.01|
|   24-Feb-05|      0.01|
+------------+----------+
only showing top 10 rows



In [53]:
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+----+----+-----+---+----+---------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+
|Name|City|State|Zip|Bank|BankState|ApprovalDate|ApprovalFY|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|GrAppv|SBA_Appv|Sector|
+----+----+-----+---+----+---------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+
+----+----+-----+---+----+---------+------------+----------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+----

In [54]:
print_unique_val_num_in_col(loan_df, col_name)

Number of unique values in ApprovalDate: 6565
Percentage of unique values in ApprovalDate: 0.73%


In [55]:

# the full date has too much detail, so we will extract the month only

loan_df = loan_df.withColumn("ApprovalMonth", split(col(col_name), "-")[1])
loan_df = loan_df.drop(col_name)
col_name = 'ApprovalMonth'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,"ApprovalMonth")


+-------------+----------+
|ApprovalMonth|Percentage|
+-------------+----------+
|          Sep|      1.03|
|          Mar|      1.03|
|          Aug|      0.99|
|          Jun|      0.99|
|          Apr|      0.98|
|          May|      0.96|
|          Jul|      0.96|
|          Oct|      0.91|
|          Dec|      0.88|
|          Nov|      0.87|
+-------------+----------+
only showing top 10 rows



 ### 10. ApprovalFY - Fiscal Year of commitment

 Drop the column as it is a date column and does not provide any information for the analysis.

In [56]:

loan_df = loan_df.drop('ApprovalFY')


 ### 11. Term - Loan term in months

In [57]:

col_name = 'Term'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+----+----------+
|Term|Percentage|
+----+----------+
|  84|      3.08|
|  60|      1.17|
| 120|      1.02|
| 240|      0.91|
| 300|      0.37|
|  12|       0.3|
|  36|      0.23|
| 180|      0.22|
|  48|      0.18|
|  72|      0.13|
+----+----------+
only showing top 10 rows



In [58]:
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+-------------+
|Name|City|State|Zip|Bank|BankState|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|GrAppv|SBA_Appv|Sector|ApprovalMonth|
+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+-------------+
+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+-------------+

Null Count: 0


In [59]:
print_unique_val_num_in_col(loan_df, col_name)

Number of unique values in Term: 330
Percentage of unique values in Term: 0.04%


In [60]:

# loan_df = loan_df.withColumn("Term_category", 
#                              when((col(col_name) <=90),'Below 3 months')
#                              .when(((col(col_name)>90) & (col(col_name)<=180)), '3-6 months')
#                              .when(((col(col_name)>180) & (col(col_name)<=365)),  '6-12 months')
#                              .otherwise('More Than a Year'))
# loan_df = loan_df.drop(col_name)
# col_name = "Term_category"
# show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)

 ### 12. NoEmp - Number of Business Employees

In [61]:

col_name = 'NoEmp'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+-----+----------+
|NoEmp|Percentage|
+-----+----------+
|    1|      1.84|
|    2|      1.77|
|    3|       1.1|
|    4|      0.84|
|    5|      0.69|
|    6|      0.51|
|    7|      0.36|
|   10|      0.36|
|    8|      0.34|
|   12|      0.24|
+-----+----------+
only showing top 10 rows



In [62]:
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+-------------+
|Name|City|State|Zip|Bank|BankState|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|GrAppv|SBA_Appv|Sector|ApprovalMonth|
+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+-------------+
+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+-------------+

Null Count: 0


In [63]:
print_unique_val_num_in_col(loan_df, col_name)

Number of unique values in NoEmp: 311
Percentage of unique values in NoEmp: 0.03%


 ### 13. NewExist - 1 = Existing business, 2 = New business

In [64]:

col_name = 'NewExist'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+--------+----------+
|NewExist|Percentage|
+--------+----------+
|       1|      8.24|
|       2|      2.95|
|       0|      0.01|
|    NULL|       0.0|
+--------+----------+



In [65]:
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+--------------------+-------------+-----+-----+--------------------+---------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------------+------------+------+-------------+
|                Name|         City|State|  Zip|                Bank|BankState|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|      GrAppv|    SBA_Appv|Sector|ApprovalMonth|
+--------------------+-------------+-----+-----+--------------------+---------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------------+------------+------+-------------+
|  Felco Products LLC|       TOTOWA|   NJ| 7512|JPMORGAN CHASE BA...|       IL|  84|    1|    NULL|        2|         

 Drop rows with 0 or Null

In [66]:

col_name = 'NewExist'
loan_df = loan_df.filter(loan_df[col_name] != 0)
loan_df = loan_df.filter(loan_df[col_name].isNotNull())
loan_df_count = loan_df.count()

In [67]:

show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+--------+----------+
|NewExist|Percentage|
+--------+----------+
|       1|     73.65|
|       2|     26.35|
+--------+----------+



Convert it to boolean, '2' is true, '1' is false.

In [68]:

loan_df = loan_df.withColumn(col_name, 
                   when(col(col_name) == "2", 1)
                   .otherwise(0)
                   .cast("int"))


In [69]:

show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+--------+----------+
|NewExist|Percentage|
+--------+----------+
|       0|     73.65|
|       1|     26.35|
+--------+----------+



 ### 14. CreateJob - Number of jobs created

In [70]:

col_name='CreateJob'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+---------+----------+
|CreateJob|Percentage|
+---------+----------+
|        0|     68.18|
|        1|      7.93|
|        2|      7.26|
|        3|      3.44|
|        4|      2.37|
|        5|      2.13|
|       10|      1.35|
|        6|      1.23|
|        8|       0.7|
|        7|      0.65|
+---------+----------+
only showing top 10 rows



In [71]:
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+-------------+
|Name|City|State|Zip|Bank|BankState|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|GrAppv|SBA_Appv|Sector|ApprovalMonth|
+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+-------------+
+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+-------------+

Null Count: 0


In [72]:
print_unique_val_num_in_col(loan_df, col_name)

Number of unique values in CreateJob: 130
Percentage of unique values in CreateJob: 0.13%


In [73]:

# loan_df = loan_df.drop(col_name)


 ### 15. RetainedJob - Number of jobs retained

In [74]:

col_name='RetainedJob'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+-----------+----------+
|RetainedJob|Percentage|
+-----------+----------+
|          0|     44.26|
|          1|      10.3|
|          2|      9.34|
|          3|      5.69|
|          4|      4.33|
|          5|      3.69|
|          6|      2.67|
|          7|       1.9|
|         10|      1.75|
|          8|      1.74|
+-----------+----------+
only showing top 10 rows



In [75]:
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+-------------+
|Name|City|State|Zip|Bank|BankState|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|GrAppv|SBA_Appv|Sector|ApprovalMonth|
+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+-------------+
+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+-------------+

Null Count: 0


In [76]:
print_unique_val_num_in_col(loan_df, col_name)

Number of unique values in RetainedJob: 218
Percentage of unique values in RetainedJob: 0.22%


In [77]:

# loan_df = loan_df.drop(col_name)


 ### 16. FranchiseCode - Franchise code, (00000 or 00001) = No franchise

In [78]:

col_name='FranchiseCode'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+-------------+----------+
|FranchiseCode|Percentage|
+-------------+----------+
|            1|      73.5|
|            0|     23.05|
|        52000|      0.21|
|        65150|       0.2|
|        50564|      0.16|
|        53650|      0.13|
|        34845|       0.1|
|        38510|       0.1|
|        34900|      0.08|
|        30811|      0.07|
+-------------+----------+
only showing top 10 rows



In [79]:
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+-------------+
|Name|City|State|Zip|Bank|BankState|Term|NoEmp|NewExist|CreateJob|RetainedJob|FranchiseCode|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|GrAppv|SBA_Appv|Sector|ApprovalMonth|
+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+-------------+
+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+-------------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+-------------+

Null Count: 0


In [80]:
print_unique_val_num_in_col(loan_df, col_name)

Number of unique values in FranchiseCode: 387
Percentage of unique values in FranchiseCode: 0.38%


 We don't care about the franchise code, we only care if there is a franchise or not

In [81]:

# make 0 or 1 = 0, anything else = 1
loan_df = loan_df.withColumn("IsFranchise", when((col(col_name) == 0) | (col(col_name) == 1), 0).otherwise(1))


In [82]:

col_name = 'IsFranchise'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+-----------+----------+
|IsFranchise|Percentage|
+-----------+----------+
|          0|     96.55|
|          1|      3.45|
+-----------+----------+



In [83]:

loan_df = loan_df.drop('FranchiseCode')


 ### 17. UrbanRural - 1 = Urban, 2 = rural, 0 = undefined

In [84]:

col_name = 'UrbanRural'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)

+----------+----------+
|UrbanRural|Percentage|
+----------+----------+
|         1|     57.66|
|         0|     27.77|
|         2|     14.57|
+----------+----------+



In [85]:
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+-------------+-----------+
|Name|City|State|Zip|Bank|BankState|Term|NoEmp|NewExist|CreateJob|RetainedJob|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|GrAppv|SBA_Appv|Sector|ApprovalMonth|IsFranchise|
+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+-------------+-----------+
+----+----+-----+---+----+---------+----+-----+--------+---------+-----------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------+--------+------+-------------+-----------+

Null Count: 0


In [86]:
print_unique_val_num_in_col(loan_df, col_name)

Number of unique values in UrbanRural: 3
Percentage of unique values in UrbanRural: 0.00%


 ### 18. RevLineCr - Revolving line of credit: Y = Yes, N = No

In [87]:

col_name = 'RevLineCr'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+---------+----------+
|RevLineCr|Percentage|
+---------+----------+
|        N|     43.21|
|        0|     30.16|
|        Y|     24.23|
|        T|      1.95|
|     NULL|      0.44|
|        R|       0.0|
|        1|       0.0|
|        `|       0.0|
|        ,|       0.0|
|        2|       0.0|
+---------+----------+



In [88]:
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+--------------------+-----------+-----+----+--------------------+---------+----+-----+--------+---------+-----------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------------+-----------+------+-------------+-----------+
|                Name|       City|State| Zip|                Bank|BankState|Term|NoEmp|NewExist|CreateJob|RetainedJob|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|      GrAppv|   SBA_Appv|Sector|ApprovalMonth|IsFranchise|
+--------------------+-----------+-----+----+--------------------+---------+----+-----+--------+---------+-----------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------------+-----------+------+-------------+-----------+
|C&G ENTERPRISES, ...|  WESTFIELD|   MA|1085|FIRST NIAGARA BAN...|       NY|  22|    4|       0|        0|          0|         0|     NU

In [89]:
print_unique_val_num_in_col(loan_df, col_name)

Number of unique values in RevLineCr: 10
Percentage of unique values in RevLineCr: 0.01%


 Filter only N and Y

In [90]:

col_name = 'RevLineCr'
print(f"Number of rows before filtering: {loan_df_count}")
loan_df = loan_df.filter(loan_df[col_name].isin('N', 'Y'))
loan_df_count = loan_df.count()
print(f"Number of rows after filtering: {loan_df_count}")
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


Number of rows before filtering: 100638
Number of rows after filtering: 67872
+---------+----------+
|RevLineCr|Percentage|
+---------+----------+
|        N|     64.07|
|        Y|     35.93|
+---------+----------+



 Transform N and Y to 0 and 1

In [91]:

loan_df = loan_df.withColumn(col_name, 
                   when(col(col_name) == "Y", 1)
                   .otherwise(0)
                   .cast("int"))


 ### 19. LowDoc - LowDoc Loan Program: Y = Yes, N = No

In [92]:

col_name = "LowDoc"
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+------+----------+
|LowDoc|Percentage|
+------+----------+
|     N|      91.3|
|     Y|       7.7|
|  NULL|      0.54|
|     S|      0.18|
|     C|      0.12|
|     0|      0.11|
|     A|      0.03|
|     R|      0.01|
+------+----------+



In [93]:
show_df_where_col_isnull(loan_df, col_name, show_num=5)

+--------------------+-----------+-----+----+--------------------+---------+----+-----+--------+---------+-----------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------------+------------+------+-------------+-----------+
|                Name|       City|State| Zip|                Bank|BankState|Term|NoEmp|NewExist|CreateJob|RetainedJob|UrbanRural|RevLineCr|LowDoc|ChgOffDate|DisbursementDate|DisbursementGross|BalanceGross|MIS_Status|ChgOffPrinGr|      GrAppv|    SBA_Appv|Sector|ApprovalMonth|IsFranchise|
+--------------------+-----------+-----+----+--------------------+---------+----+-----+--------+---------+-----------+----------+---------+------+----------+----------------+-----------------+------------+----------+------------+------------+------------+------+-------------+-----------+
|        Jose Sanchez|NORTHAMPTON|   MA|1060|FLORENCE SAVINGS ...|       MA|  60|    0|       1|        1|          0|         1|    

In [94]:
print_unique_val_num_in_col(loan_df, col_name)

Number of unique values in LowDoc: 8
Percentage of unique values in LowDoc: 0.01%


 Filter only N and Y

In [95]:

col_name = 'LowDoc'
print(f"Number of rows before filtering: {loan_df_count}")
loan_df = loan_df.filter(loan_df[col_name].isin('N', 'Y'))
loan_df_count = loan_df.count()
print(f"Number of rows after filtering: {loan_df_count}")
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


Number of rows before filtering: 67872
Number of rows after filtering: 67192
+------+----------+
|LowDoc|Percentage|
+------+----------+
|     N|     92.22|
|     Y|      7.78|
+------+----------+



 Transform N and Y to 0 and 1

In [96]:

loan_df = loan_df.withColumn(col_name, 
                   when(col(col_name) == "Y", 1)
                   .otherwise(0)
                   .cast("int"))


 ### 20. ChgOffDate - The date when a loan is declared to be in default

 Drop the column due to the high number of missing values.

In [97]:

loan_df = loan_df.drop('ChgOffDate')


 ### 21. DisbursementDate - Date when loan was disbursed

In [98]:

loan_df = loan_df.drop('DisbursementDate')


 ### 22. DisbursementGross - Amount disbursed

In [99]:

loan_df = loan_df.drop('DisbursementGross')


 ### 23. BalanceGross - Gross amount outstanding

In [100]:

col_name = 'BalanceGross'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+------------+----------+
|BalanceGross|Percentage|
+------------+----------+
|      $0.00 |     100.0|
| $84,617.00 |       0.0|
|$996,262.00 |       0.0|
+------------+----------+



 Drop as most of the values are 0

In [101]:

loan_df = loan_df.drop('BalanceGross')


 ### 24. MIS_Status - Target variable

 Delete rows that have null target value (MIS_Status)

In [102]:

col_name ="MIS_Status"
show_percentage_of_each_value_in_column(loan_df, loan_df_count, col_name)


+----------+----------+
|MIS_Status|Percentage|
+----------+----------+
|     P I F|     81.18|
|    CHGOFF|     18.51|
|      NULL|      0.31|
+----------+----------+



In [103]:

# drop rows with null values in MIS_Status column
loan_df = loan_df.dropna(subset=[col_name])
show_percentage_of_each_value_in_column(loan_df, loan_df_count, col_name)



+----------+----------+
|MIS_Status|Percentage|
+----------+----------+
|     P I F|     81.18|
|    CHGOFF|     18.51|
+----------+----------+



 ### Replace target values with 0 and 1

 Target value column is: MIS_Status

 "P I F" = 1

 "CHGOFF" = 0

In [104]:

loan_df = loan_df.withColumn(col_name, 
                   when(col(col_name) == "P I F", 1)
                   .otherwise(0)
                   .cast("int"))


 Show the percentage of:

 - Paid in full loans (approved loans), MIS_Status = 1

 - Charged off loans (rejected loans), MIS_Status = 0

In [105]:

show_percentage_of_each_value_in_column(loan_df, loan_df_count, col_name)


+----------+----------+
|MIS_Status|Percentage|
+----------+----------+
|         1|     81.18|
|         0|     18.51|
+----------+----------+



 ### 25. ChgOffPrinGr - Charged-off amount

In [106]:

col_name = 'ChgOffPrinGr'
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+------------+----------+
|ChgOffPrinGr|Percentage|
+------------+----------+
|      $0.00 |     80.73|
| $50,000.00 |      0.31|
| $10,000.00 |      0.25|
| $25,000.00 |      0.21|
| $35,000.00 |      0.18|
|$100,000.00 |      0.14|
| $20,000.00 |      0.08|
| $15,000.00 |      0.07|
|  $5,000.00 |      0.06|
| $30,000.00 |      0.05|
+------------+----------+
only showing top 10 rows



 Drop this column as it will leak info to the column, because if the value is 0, this means that the loan is charged off

In [107]:

loan_df = loan_df.drop('ChgOffPrinGr')


 ### 26. GrAppv - Gross amount of loan approved by bank

In [108]:

col_name = "GrAppv"
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+------------+----------+
|      GrAppv|Percentage|
+------------+----------+
| $50,000.00 |      8.91|
| $25,000.00 |      6.99|
|$100,000.00 |      6.31|
| $10,000.00 |      5.25|
| $35,000.00 |       3.3|
| $20,000.00 |      3.16|
|$150,000.00 |      2.69|
| $30,000.00 |      2.57|
| $15,000.00 |      2.45|
| $75,000.00 |      2.24|
+------------+----------+
only showing top 10 rows



 #### Clean this column

 - Remove $

 - Remove ,

 - Convert to float

In [109]:

loan_df = loan_df.withColumn("clean_GrAppv", regexp_replace("GrAppv", "\$", ""))  # Remove $
loan_df = loan_df.withColumn("clean_GrAppv", regexp_replace("clean_GrAppv", ",", ""))  # Remove comma
loan_df = loan_df.withColumn("clean_GrAppv", col("clean_GrAppv").cast("float"))
col_name = "clean_GrAppv"
show_percentage_of_each_value_in_column(loan_df, loan_df_count,col_name)


+------------+----------+
|clean_GrAppv|Percentage|
+------------+----------+
|     50000.0|      8.91|
|     25000.0|      6.99|
|    100000.0|      6.31|
|     10000.0|      5.25|
|     35000.0|       3.3|
|     20000.0|      3.16|
|    150000.0|      2.69|
|     30000.0|      2.57|
|     15000.0|      2.45|
|     75000.0|      2.24|
+------------+----------+
only showing top 10 rows



In [110]:

loan_df = loan_df.drop('GrAppv')


 ### 27. SBA_Appv - SBA's guaranteed amount of approved loan

 Drop as we don't know this amount in the future

In [111]:

loan_df = loan_df.drop('SBA_Appv')


 ### Final schema

In [112]:

loan_df.printSchema()


root
 |-- Name: string (nullable = false)
 |-- City: string (nullable = false)
 |-- State: string (nullable = false)
 |-- Zip: string (nullable = true)
 |-- Bank: string (nullable = false)
 |-- BankState: string (nullable = true)
 |-- Term: integer (nullable = true)
 |-- NoEmp: integer (nullable = true)
 |-- NewExist: integer (nullable = false)
 |-- CreateJob: integer (nullable = true)
 |-- RetainedJob: integer (nullable = true)
 |-- UrbanRural: integer (nullable = true)
 |-- RevLineCr: integer (nullable = false)
 |-- LowDoc: integer (nullable = false)
 |-- MIS_Status: integer (nullable = false)
 |-- Sector: integer (nullable = true)
 |-- ApprovalMonth: string (nullable = true)
 |-- IsFranchise: integer (nullable = false)
 |-- clean_GrAppv: float (nullable = true)



 ### Check duplicated rows based on all columns



In [113]:

print("Number of duplicate rows in the dataframe:")
loan_df_duplicates = loan_df_count - loan_df.dropDuplicates().count()
print(loan_df_duplicates)
loan_df = loan_df.dropDuplicates()

Number of duplicate rows in the dataframe:


238


 ### Final DF Count

In [114]:

loan_df_count = loan_df.count()
print(f"Final DF count: {loan_df_count}")


Final DF count: 66954


In [115]:
if print_reports:
    report_res = report_df(loan_df, loan_df.columns)
    # Display the result
    column_names = ['Column', 'Type', 'Unique Sample', 'N Unique', '%None']
    print(tabulate(report_res, headers=column_names, tablefmt='grid'))

In [116]:
# output_path = "../data/preprocessed.csv"

# # Save the DataFrame to a CSV file
# loan_df.write.csv(output_path, header=True, mode="overwrite")

In [117]:
# Convert PySpark DataFrame to Pandas DataFrame
pandas_df = loan_df.toPandas()

# Specify the path where you want to save the CSV file
output_path = "../data/preprocessed.csv"

# Save the Pandas DataFrame to a CSV file
pandas_df.to_csv(output_path, index=False)

In [121]:
sample_size = 1000
# Save a sample
output_path = f"../sample_data/{sample_size}.csv"

# Save the first 50000 rows of the Pandas DataFrame to a CSV file
pandas_df.head(sample_size).to_csv(output_path, index=False)